In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab06.ipynb")

In [ ]:
from datascience import *
import numpy as np

# Lab 6 – Complex Table Methods

## Data 94, Spring 2021

Today we will be exploring some more complex table functions we can use! The `apply`, `group`, `pivot`, and `join` methods all allow us to perform different queries, both directly and indirectly. Understanding how each one works is important to being able to use them properly, and by the end of this lab we will be able to query tables in some pretty cool ways!

These new methods allow us to do different operations than before, and it is becoming more and more important as we learn new table methods to remember how each method sits on our Data Science toolbelt. We should think of each new method as a new tool that serves a specific purpose, and your job as a data scientist is not only to understand what each tool does, but when each tool is applicable in new situations!

Let's load in a dataset and see what we can do with these new methods:

In [ ]:
movies = Table.read_table("data/movies.csv")

# We don't need the Profitability of this dataset, so we drop it before we start
# This is an aspect of data cleaning you will get used to over time
# Most datasets have extra information you don't necessarily need
movies = movies.drop("Profitability")

movies.show(5)

## The [apply](http://data8.org/datascience/_autosummary/datascience.tables.Table.apply.html#datascience.tables.Table.apply) method

The `apply` method allows us to map a function's behavior onto an entire column of a table. We can define functions in a notebook and then *apply* those functions to the columns of a table.

The `apply` method takes at least 2 arguments. The first is a function, and the rest are as many column labels you need to run that function. If the function you provide needs 2 inputs, you need to list 2 columns for it to work on, etc.

`apply` returns a NumPy array of the transformed values, so the `apply` method is a query on its own. We can ask questions like "How can I categorize the items in this column?" (like converting grade percentages into letter grades from lecture). We can also make modifications to a table, like rounding all the values in a column to a certain accuracy.

We can see some examples of `apply` at work with functions we can make ourselves:

In [ ]:
# We can define a function that indicates how good a movie is based on it's "Audience score"
# Here, the input variable "audience_score" has a type of "int", 
# so we will have to provide "movie_quality" a column of "ints"
def movie_quality(audience_score):
    if audience_score > 80:
        return "Great"
    elif audience_score > 60:
        return "Good"
    elif audience_score > 30:
        return "Okay"
    else:
        return "Bad"

In [ ]:
# We apply our "movie_quality" function to the "movies" table
qualities = movies.apply(movie_quality, "Audience score %")
qualities

We now have a NumPy array of all the movie qualities. We can add this array as a column of our table using the `with_column` method!

In [ ]:
movies_with_qualities = movies.with_column("Quality", qualities)
movies_with_qualities.show(5)

We can use `apply` in this way to add columns to a table that transforms something in the table to something that more people can understand more easily!

### Input Type and Output Type

Very quickly, we should talk about **input type** and **output type**. **Input type** is the *type* of the arguments in a function. A function can take in as input any of the types we have talked about so far in this class (int, str, boolean, etc.). The input type is very important when using the table methods we talk about today, because there can be behavior we do not expect if we give a function the wrong input type. Let's see an example of this behavior when we try to use our `movie_quality` function on a row that has an **input type** of `str` instead of `int`:

In [ ]:
movies.apply(movie_quality, "Genre")

The error message does not make it *explicitly* clear that the function you provided got an incorrect input type, but if you are using a table method and the error message seems to indicate incompatible types interacting with each other, that may be the issue.

## The [group](http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html#datascience.tables.Table.group) method

The `group` method is very helpful for organizing a table before asking more questions about it. you can think of the `group` method as organizing rows into bins based on one of their values. All the rows that share a column value go in the same bin!

`group` takes in 1-2 arguments. The first is a column label to group by, and the second is an optional function argument to group on, which defaults to counting the number of rows in the bin. We will see some examples of how this optional argument works below.

Let's use the `group` method to organize our movies by `Genre` so that we can see which genres are in our table:

In [ ]:
movies_by_genre = movies.group("Genre")
movies_by_genre

As you can see there are some `Genres` that are close to another `Genre`, but Python is not smart enough to know that! This is common in Data Science, and taking care of misspellings is a huge part of *cleaning* data before we use it. We won't cover that in this lab, but it is important to note this when working with datasets you did not collect yourself (which is most of the time). You can feel free to check out what this function is doing to clean the `Genre` column!

In [ ]:
# Do not change this cell, just run it
def clean_genre(table):
    genre = table.column("Genre")
    cleaned_genres = []
    for g in genre:
        if g in ["Comdy", "comedy"]:
            cleaned_genres.append("Comedy")
        elif g in ["Romence", "romance"]:
            cleaned_genres.append("Romance")
        else:
            cleaned_genres.append(g)
    return table.with_column("Genre", np.array(cleaned_genres))

movies_with_qualities = clean_genre(movies_with_qualities)
movies = clean_genre(movies)
movies_with_qualities.group("Genre")

What if we want to group by movie *quality*? Well we have our `movies_with_qualities` table that we made earlier with the `Quality` information to use:

In [ ]:
quality_groups = movies_with_qualities.group("Quality")
quality_groups

We can use the optional function argument to ask more complex questions instead of just asking for the number of rows in each bin. For example, to see the average `Audience Score` of each bin, we can use the `np.mean` or `np.average` functions (for our purposes, they are exactly the same, and you can use them interchangeably in this class) to group *on* `Quality`: 

In [ ]:
quality_group_ratings = movies_with_qualities.group("Quality", np.average)
quality_group_ratings

We cannot specify which column we want the average of, so as you can see Python takes the average of every column. For rows where it is possible to take the average (columns with the correct input type), it does so. But in columns where taking the average doesn't work (with strings in this case), Python outputs nothing for those column averages.

Also, the column labels in this new table now have `average` at the end **except** the column label you grouped on. The name of the function you choose to group on will appear at the end of every label in the resulting table. **Like most table methods, the original table you are grouping does not change unless you reassign it**.

To get only the column we wanted the average of, we can use the `select` method to get the `Quality` and the `Audience score % average`:

In [ ]:
quality_ratings = quality_group_ratings.select("Quality", "Audience score % average")
quality_ratings

The table defaults to sort on the column you grouped on, which in this case is alphabetical order for strings. If we want to see the rows in order of average audience score, we can use the `sort` function:

In [ ]:
quality_ratings.sort("Audience score % average", descending=True)

## Question 1 – Do Well-Liked Movies Make Money?

We can expand on this query even more if we want to know *how much money* each quality level made, on average. We can use the `group` method to do this by passing in a function that takes the average of an array:

In [ ]:
# Do not change this cell, just run it
def clean_worldwide_gross(table):
    return table.with_column("Worldwide Gross (Millions)", table.apply(lambda x: float(x[1:]), "Worldwide Gross"))

movies_with_qualities = clean_worldwide_gross(movies_with_qualities)
movies = clean_worldwide_gross(movies)
movies.show(5)

We cleaned the `Worldwide Gross` column for you because it was a column of **strings** to accommodate the dollar sign. This is why the Worldwide Gross column did not appear in the `group` call we did above. **The column label we are interested in is now `Worldwide Gross (Millions)`**.

Assign `group_label` to the **column label** we want to group on to put the rows in bins based on movie quality.

Assign `function` to a **function** you know that we use while grouping to get the average money grossed by movie quality.

Assign `new_column_label` to the **new column label** that will contain the average `Worldwide Gross` values after our correct `group` call.

<!--
BEGIN QUESTION
name: q1
points: 0
-->

In [ ]:
group_label = ...
function = ...
new_column_label = ...

money_by_quality = movies_with_qualities.group(group_label, function).select(group_label, new_column_label)
money_by_quality

In [ ]:
grader.check("q1")

## The [pivot](http://data8.org/datascience/_autosummary/datascience.tables.Table.pivot.html#datascience.tables.Table.pivot) method

The `pivot` method allows us to see the *intersection* of two of our column labels. `Pivot` essentially sorts the contents of the dataset based on the combination of the two column labels you pivot on. All the table's rows that share values in the pivoting columns all go into the same bin, and this happens for all combinations of the first and second column you are pivoting on.

The `pivot` method has 4 important arguments, 2 of which are mandatory and 2 of which are optional:

| **Argument** | **Description** |
| --- | --- |
| `columns` | The label whose unique values will appear as the **columns** of the outputted pivot table |
| `rows` | The label whose unique values will appear as the **rows** of the outputted pivot table |
| *Optional:* `values` | Values to use when aggregating |
| *Optional:* `collect` | Function used to aggregate the `values` provided in the previous argument |

You must use the two `values` and `collect` arguments together, one does not work without the other.

The best way to understand how `pivot` works is to look at some examples and talk about what happens. Here we will pivot the `movies_with_qualities` table on its `Genre` and `Quality` column labels to see how many of each type of *genre* are in each category of *quality*:

The way we can read this table is almost how we read a graph in a math class. To see how many **Good Comedies** there are, we look at the row corresponding to `Good` and the column corresponding to `Comedy`, so there are 18 **Good Comedies**.

The default behavior of `pivot` is to just count the rows that appear at each intersection of the pivot. However, we can ask it to count another value in the table using any function we want! For example, if we want to know **how much money** each intersection made on average instead of **how many** movies appear in each intersection, we can do that as well!

In [ ]:
genre_quality_average_grosses = movies_with_qualities.pivot("Genre", "Quality", values="Worldwide Gross (Millions)", collect=np.average)
genre_quality_average_grosses

Based on the small dataset we have in this notebook, it seems that a **Great Animation** movie is bound to net you the most money with a whopping 438 million dollars on average! It is important to remember, however, that the dataset we are working with only has 3 animation films, so that may not be the best representation of animation films. If you do work on larger, less biased datasets, you can usually draw more concrete conclusions, but nevertheless it is cool to see the `pivot` function at work!

## Question 2 – Does Studio Matter?

Now, let's write a query that can tell us what genre of movie each studio tends to make, and how much those movies made in total. We can use the `pivot` method just like we did above to see the breakdown of movies by studio and genre, and then aggregate those rows by adding up the worldwide gross of each intersection!

Assign `column_label` and `row_label` to column labels of `movies_with_qualities` that make the resulting `pivot` call have studios as the rows and genres as the columns.

Assign `value_to_collect` to a column label that we can use to collect the data we need to ultimately see the total amount of money made by each intersection of genre and studio.

Assign `collection_function` to a function you know that can add up all the movie earnings at each intersection (*Hint: it is a NumPy function*).

<!--
BEGIN QUESTION
name: q2
points: 0
-->

In [ ]:
column_label = ...
row_label = ...
value_to_collect = ...
collection_function = ...

studio_genre_total_gross = movies_with_qualities.pivot(column_label, row_label, value_to_collect, collection_function)
studio_genre_total_gross

In [ ]:
grader.check("q2")

## The [join](http://data8.org/datascience/_autosummary/datascience.tables.Table.join.html#datascience.tables.Table.join) method

The last method we will talk about in this lab is the `join` method. This method allows us to combine two different tables together!

The `join` method takes in 2 mandatory arguments and 1 optional argument:

| **Column Name** | **Description** |
| --- | --- |
| `column_label` | a column to use to join |
| `other` | another table |
| *Optional:* `other_label` | `other`'s label to join on (if not the same as `column_label`) |

If `other` has a label in common with the table you are joining with and this common label is the one you want to join on, then you do not need to use the optional argument. If you want to join on another column label or if neither table has a column label in common, then you can use the optional `other_label`.

The way join works takes some getting used to, so let's look at some examples of `join` at work!

We have the `dogs` and `owners` tables below, take a look at them a bit before we move on so you understand what data they contain:

In [ ]:
dogs = Table().with_columns(
    "Name", np.array(["Junior", "Spot", "Fluffy", "Doge"]),
    "Breed", np.array(["Cockapoo", "Golden Retriever", "Corgi", "Doge"]),
    "Owner", np.array(["Suraj", "Isaac", "Angela", "Oski"]),
)
dogs

In [ ]:
owners = Table().with_columns(
    "Owner", np.array(["Suraj", "Isaac", "Angela", "Oski"]),
    "Owner Age", np.array([22, 21, 21, 153])
)
owners

As you can see, we have a column in common: `Owner`. Let's join these two tables together so that we can have all the doggy data in one place!

In [ ]:
doggy_data = dogs.join("Owner", owners)
doggy_data

This table now has all of our information in one place, which makes using it easier!

Now let's take a look at a more common example, where the column labels being named differently can cause a problem. We will use the exact same `dogs` and `owners` tables, but we will change a column label on `owners`:

In [ ]:
owners_new_label = owners.relabeled("Owner", "Name")

display(dogs, owners_new_label)

Now if we try to use the `join` method like we did last time, we run into an issue...

In [ ]:
doggy_data = dogs.join("Owner", owners_new_label)
doggy_data

Because the `owners` table does not have a column label called `Owner`, we may try to use the one column label they do have in common: `Name`...

In [ ]:
doggy_data = dogs.join("Name", owners_new_label)
doggy_data

...but this doesn't appear to work either. This is not an error, there is simply no table outputted by this join call!

This `join` call does not do what we want, because the `dogs` `Name` corresponds to the **dog's** name, but the `owners` `Name` corresponds to the **Owner's** name! No dog and owner have any of the same names, so there is no data to output in this `join` call!

Instead, we have to make sure we join on the `Owner` column from `dogs` and the `Name` column from `owners`! We can do this using the third *optional* argument in the `join` method:

In [ ]:
doggy_data = dogs.join("Owner", owners_new_label, "Name")
doggy_data

In this table, the `Name` column now refers to the name of the dog, and the `Owner` column corresponds to the name of the owner!

## Question 3 – A Slightly Different `Join`

The `join` method can also change the number of rows in its output. If there are multiple rows in one table that match with one row in the other, the `join` method will include rows for each of these matches in the output. Also, if there is a row in a table with no match in the other, there will be no row in the output that represents this row. Let's implement both these situations in practice and see how they work:

In [ ]:
# Just run this cell
# This new dogs table has a new extra dogs
more_dogs = dogs.with_rows(np.array([np.array(["Senior", "Cockapoo", "Suraj"]), np.array(["Doug", "Golden Retriever", "Russell"])]))
more_dogs

Before we exectute the join between these tables, we should be able to calculate how many rows should there be in the output. Assign the variable `more_dog_owner_rows` to the number of rows that should result from joining `more_dogs` with `owners_new_label`. Run the cell below to see them again for clarity:

In [ ]:
display(more_dogs, owners_new_label)

*Remember, each row in the `more_dogs` table only gets a row in the output if it matches a row in `owners_new_label`*

<!--
BEGIN QUESTION
name: q3
points: 0
-->

In [ ]:
more_dog_owner_rows = ...

In [ ]:
grader.check("q3")

In [ ]:
# If an owner in the table has 2 dogs, both dogs should appear in the output
# If a dog has no owner in the owners table, the dog does not appear in the output
complex_doggy_data = more_dogs.join("Owner", owners_new_label, "Name")
complex_doggy_data

In [ ]:
# Doug is not in the new table, and both Suraj's dogs are present for a total of 5 rows
complex_doggy_data.num_rows

As we expect, both `Junior` and `Senior` appear as Suraj's dogs, but `Doug` does not appear in the table (fear not, he still has an owner, but `Russell` is not in the owners table).

## Done! 😇

That's it! There's nowhere for you to submit this, as labs are not assignments. However, please ask any questions you have with this notebook in lab or on Ed.

There are no extra problems this week, good luck with the homework!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)